# 라이브러리 불러오기

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
     
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from pandas.io.html import read_html
import time
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from datetime import datetime, timedelta

# Scrapping(Departure)

In [2]:
#중앙 데이터프레임 만들기
df_delay_reason = pd.DataFrame(columns=['항공사','목적지','계획','예상','출발','구분','현황','비정상원인','날짜'])

In [3]:
#날짜 리스트 생성
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("20070101", "20210731")

In [4]:
driver = webdriver.Chrome(executable_path='/Users/ikgyu/datacamp/project/chromedriver')

#암묵적으로 웹 자원 로드를 위해 10초까지 기다려 준다.
driver.implicitly_wait(10)

#url 접근
driver.get('https://www.airportal.go.kr/life/airinfo/RbBejFrm.jsp#')
    
#'출발' click radio button
time.sleep(10)
accept_1 = driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/input[2]")
accept_1.click()
#안 될때, Kernel -> Restart & Run all

In [5]:
for num in range(0,len(dates)):
    
    #execute_script 함수 사용
    driver.execute_script("document.getElementsByName('current_date')[0].value=\'"+dates[num]+"\'")
    
    #'검색' click
    accept_2 = driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[6]/a/img")
    accept_2.click()
    
    #iframe 부르기
    iframes = driver.find_elements_by_tag_name('iframe')
    driver.switch_to.frame(iframes[0])
    html = driver.page_source
    
    #원래 프레임으로 돌아가기
    driver.switch_to.default_content()

    #html parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    #매개 데이터프레임 선언
    df_sample = pd.DataFrame()
    
    #요소별 리스트 생성
    airline_list = []
    destination_list = []
    plan_list = []
    prediction_list = []
    Start_list = []
    division_list = []
    Status_list = []
    Abnormal_cause_list = []

    #항공사 추출
    for airline in soup.find_all("td", {"width":"140"}):
        airline_list.append(airline.text.strip())
    
    #목적지 추출
    for destination in soup.find_all("td", {"width":"80"})[1::2]:
        destination_list.append(destination.text)
    
    #계획, 예상, 출발, 구분, 현황 추출
    list_ = []
    for info in soup.find_all("td", {"width":"50"}):
        list_.append(info.text)
    for i in range(0, len(list_)):
        if((i%5)==0):
            plan_list.append(list_[i])
        elif((i%5)==1):
            prediction_list.append(list_[i])
        elif((i%5)==2):
            Start_list.append(list_[i])
        elif((i%5)==3):
            division_list.append(list_[i])
        else:
            Status_list.append(list_[i])

    #비정상원인 추출
    for Abnormal_cause in soup.find_all("td", {"width":"100"}):
        Abnormal_cause_list.append(Abnormal_cause.text.strip())

    df_sample['항공사'] = Series(airline_list)
    df_sample['목적지'] = Series(destination_list)
    df_sample['계획'] = Series(plan_list)
    df_sample['예상'] = Series(prediction_list)
    df_sample['출발'] = Series(Start_list)
    df_sample['구분'] = Series(division_list)
    df_sample['현황'] = Series(Status_list)
    df_sample['비정상원인'] = Series(Abnormal_cause_list)
    df_sample['날짜'] = dates[num]
    
    #중앙 데이터프레임에 합치기
    df_delay_reason = pd.concat([df_delay_reason,df_sample])
    
df_delay_reason

<ipython-input-5-7ede68e908d5>:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['항공사'] = Series(airline_list)
<ipython-input-5-7ede68e908d5>:63: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['목적지'] = Series(destination_list)
<ipython-input-5-7ede68e908d5>:64: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['계획'] = Series(plan_list)
<ipython-input-5-7ede68e908d5>:65: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['예상'] = Series(prediction_list)
<ipython-input-5-7ede68e90

,항공사,목적지,계획,예상,출발,구분,현황,비정상원인,날짜
0,중국동방항공,PVG(푸동),08:55,:,:,여객,취소,연결-항공기,20070101
1,대한항공,SHE(심양),11:15,12:44,12:40,여객,지연,기상-시정,20070101
2,중국국제항공,TSN(천진),12:20,14:32,14:26,여객,지연,연결-항공기,20070101
3,중국동방항공,XIY(시안),12:45,14:06,14:02,여객,지연,연결-항공기,20070101
4,중국동방항공,TAO(청도),15:10,16:23,16:20,여객,지연,기타,20070101
...,...,...,...,...,...,...,...,...,...
34,칼리타화물항공,ANC(앵커리지),15:40,19:45,20:25,화물,지연,,20210731
35,칼리타화물항공,ANC(앵커리지),17:45,21:30,21:42,화물,지연,연결-항공기,20210731
36,서던화물항공,HKG(홍콩),21:30,23:45,23:58,화물,지연,,20210731
37,원통화물항공,YNZ(옌청),21:35,:,:,화물,취소,연결-항공기,20210731


In [6]:
df_delay_reason.to_csv('Incheon_Departure_DelayReason_Crawling.csv',encoding='utf-8-sig')

# Scrapping(Arrival)

In [2]:
#중앙 데이터프레임 만들기
df_delay_reason = pd.DataFrame(columns=['항공사','출발지','계획','예상','도착','구분','현황','비정상원인','날짜'])

In [3]:
#날짜 리스트 생성
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("20070101", "20210731")

In [4]:
driver = webdriver.Chrome(executable_path='/Users/ikgyu/datacamp/project/chromedriver')

#암묵적으로 웹 자원 로드를 위해 10초까지 기다려 준다.
driver.implicitly_wait(10)

#url 접근
driver.get('https://www.airportal.go.kr/life/airinfo/RbBejFrm.jsp#')
    
#'도착' click radio button -> Default
time.sleep(10)
#안 될때, Kernel -> Restart & Run all

In [5]:
for num in range(0,len(dates)):
    
    #execute_script 함수 사용
    driver.execute_script("document.getElementsByName('current_date')[0].value=\'"+dates[num]+"\'")
    
    #'검색' click
    accept_2 = driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[6]/a/img")
    accept_2.click()
    
    #iframe 부르기
    iframes = driver.find_elements_by_tag_name('iframe')
    driver.switch_to.frame(iframes[0])
    html = driver.page_source
    
    #원래 프레임으로 돌아가기
    driver.switch_to.default_content()

    #html parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    #매개 데이터프레임 선언
    df_sample = pd.DataFrame()
    
    #요소별 리스트 생성
    airline_list = []
    DeparturePoint_list = []
    plan_list = []
    prediction_list = []
    Arrival_list = []
    division_list = []
    Status_list = []
    Abnormal_cause_list = []

    #항공사 추출
    for airline in soup.find_all("td", {"width":"140"}):
        airline_list.append(airline.text.strip())
    
    #출발지 추출
    for departure in soup.find_all("td", {"width":"80"})[1::2]:
        DeparturePoint_list.append(departure.text)
    
    #계획, 예상, 도착, 구분, 현황 추출
    list_ = []
    for info in soup.find_all("td", {"width":"50"}):
        list_.append(info.text)
    for i in range(0, len(list_)):
        if((i%5)==0):
            plan_list.append(list_[i])
        elif((i%5)==1):
            prediction_list.append(list_[i])
        elif((i%5)==2):
            Arrival_list.append(list_[i])
        elif((i%5)==3):
            division_list.append(list_[i])
        else:
            Status_list.append(list_[i])

    #비정상원인 추출
    for Abnormal_cause in soup.find_all("td", {"width":"100"}):
        Abnormal_cause_list.append(Abnormal_cause.text.strip())

    df_sample['항공사'] = Series(airline_list)
    df_sample['출발지'] = Series(DeparturePoint_list)
    df_sample['계획'] = Series(plan_list)
    df_sample['예상'] = Series(prediction_list)
    df_sample['도착'] = Series(Arrival_list)
    df_sample['구분'] = Series(division_list)
    df_sample['현황'] = Series(Status_list)
    df_sample['비정상원인'] = Series(Abnormal_cause_list)
    df_sample['날짜'] = dates[num]
    
    #중앙 데이터프레임에 합치기
    df_delay_reason = pd.concat([df_delay_reason,df_sample])
    
df_delay_reason

<ipython-input-5-77892aaa9768>:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['항공사'] = Series(airline_list)
<ipython-input-5-77892aaa9768>:63: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['출발지'] = Series(DeparturePoint_list)
<ipython-input-5-77892aaa9768>:64: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['계획'] = Series(plan_list)
<ipython-input-5-77892aaa9768>:65: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['예상'] = Series(prediction_list)
<ipython-input-5-77892a

,항공사,출발지,계획,예상,도착,구분,현황,비정상원인,날짜
0,싱가폴항공,SIN(싱가포르),01:40,01:33,01:39,여객,지연,기상-시정,20070101
1,싱가폴항공,SIN(싱가포르),01:40,01:33,01:39,여객,지연,기상-시정,20070101
2,칼리타화물항공,HKG(홍콩),02:30,09:29,09:29,화물,지연,기타,20070101
3,서던화물항공,HKG(홍콩),23:30,23:30,:,화물,취소,,20070101
0,대한항공,TLV(텔아비브),00:25,02:57,02:57,화물,지연,기타,20070102
...,...,...,...,...,...,...,...,...,...
30,델타항공,ATL(애틀랜타),17:50,18:59,18:59,여객,지연,연결-항공기,20210731
31,아메리칸항공,PVG(푸동),20:05,22:31,22:31,화물,지연,연결-항공기,20210731
32,캐세이퍼시픽항공,HKG(홍콩),20:25,21:55,21:55,화물,지연,연결-항공기,20210731
33,원통화물항공,YNZ(옌청),20:35,:,:,화물,취소,연결-항공기,20210731


In [6]:
df_delay_reason.to_csv('Incheon_Arrival_DelayReason_Crawling.csv',encoding='utf-8-sig')